In [ ]:
import importlib

import torch

In [ ]:
# Test whether progress bars work.
from tqdm.auto import tqdm
_ = list(tqdm(range(1)))

## Prepare dataset

In [ ]:
from awe.data import qa_dataset, swde

_ = importlib.reload(qa_dataset)

In [ ]:
sds = swde.Dataset(suffix='-exact')
pages = [p for w in sds.verticals[0].websites for p in w.pages]

In [ ]:
#qa_dataset.prepare_entries(pages, skip_existing=True)

In [ ]:
loader = qa_dataset.QaEntryLoader(pages)

In [ ]:
#loader.validate()

In [ ]:
entry = loader[9999]
entry.id

In [ ]:
entry.get_all_answer_spans()

In [ ]:
entry.labels

## Invoke Transformer

In [ ]:
from awe import qa_model
importlib.reload(qa_model)
pipeline = qa_model.QaPipeline()

In [ ]:
pipeline.load()

In [ ]:
importlib.reload(qa_dataset)
ds = qa_dataset.QaTorchDataset(loader, pipeline.tokenizer)

In [ ]:
#ds.validate_all()

In [ ]:
idx = 1
_, label, values = ds.at(idx)
label, values

In [ ]:
encodings = ds[idx]
encodings

In [ ]:
outputs = pipeline.model(**encodings)
#outputs

In [ ]:
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits)
answer_start, answer_end

In [ ]:
pipeline.tokenizer.decode(encodings['input_ids'][0, answer_start:answer_end + 1])

### Train Transformer

In [ ]:
from awe import qa_model
importlib.reload(qa_model)
model = qa_model.QaModel(pipeline)

In [ ]:
importlib.reload(qa_dataset)
ds = qa_dataset.QaTorchDataset(loader, pipeline.tokenizer)

In [ ]:
#ds.validate_all()

In [ ]:
import torch.utils.data
import pytorch_lightning as pl

In [ ]:
train_loader = torch.utils.data.DataLoader(ds, batch_size=1, shuffle=True)
val_loader = torch.utils.data.DataLoader(ds, batch_size=1)

In [ ]:
from awe import gym
g = gym.Gym(None, None, version_name='')

In [ ]:
trainer = pl.Trainer(
    max_epochs=1,
    logger=g.create_logger(),
)

In [ ]:
trainer.validate(model, val_loader)